# Tree of Thouht (ToT) example

The Tree of Thought (ToT) is a chain that allows you to query a Large Language Model (LLM) using the Tree of Thought technique. This is based on the papaer ["Large Language Model Guided Tree-of-Thought"](https://arxiv.org/pdf/2305.08291.pdf)

In [1]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=1, max_tokens=200, verbose=True)

In [2]:
sudoku_puzzle =   "3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1"
sudoku_solution = "3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1"
problem_description = f"""
We have a 4x4 Sudoku puzzle \"{sudoku_puzzle}\" where * represents a cell to be filled.

Additional Instructions:

- Use digits 1-4.
- Output the <next_step> in the same format.
- Fill only one cell at each step.
- A partial but correct solution such as \"{sudoku_puzzle}\" is preferable to a full but incorrect one.
"""
print(problem_description)


We have a 4x4 Sudoku puzzle "3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1" where * represents a cell to be filled.

Additional Instructions:

- Use digits 1-4.
- Output the <next_step> in the same format.
- Fill only one cell at each step.
- A partial but correct solution such as "3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1" is preferable to a full but incorrect one.



In [3]:
from langchain.chains.tot.base import ToTChecker, SolutionType
import re

class MyChecker(ToTChecker):
    def evaluate(self, problem_description: str, solution: str) -> SolutionType:
        clean_solution = solution.replace(" ", "")
        regex_solution = clean_solution.replace("*", ".").replace("|", "\\|")
        if sudoku_solution in clean_solution:
            return SolutionType.VALID_FINAL
        elif re.search(regex_solution, sudoku_solution):
            return SolutionType.VALID_INTERMEDIATE
        else:
            return SolutionType.INVALID

Just testing the MyChecker class above:

In [4]:
checker = MyChecker()
assert checker.evaluate("", "3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1") == SolutionType.VALID_INTERMEDIATE
assert checker.evaluate("", "3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1") == SolutionType.VALID_FINAL
assert checker.evaluate("", "3,4,1,2|1,2,3,4|2,1,4,3|4,3,*,1") == SolutionType.VALID_INTERMEDIATE
assert checker.evaluate("", "3,4,1,2|1,2,3,4|2,1,4,3|4,*,3,1") == SolutionType.INVALID

Initialize and run the ToT chain, with maximum number of interactions `k` set to `10`.

In [5]:
from langchain.chains.tot.base import ToTChain

tot_chain = ToTChain(llm=llm, checker=MyChecker(), k=10, verbose=True)
tot_chain.run({"problem_description": problem_description})



> Entering new ToTChain chain...
Starting the ToT solve procedure.


> Entering new LLMChain chain...
Prompt after formatting:

For the given problem:


We have a 4x4 Sudoku puzzle "3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1" where * represents a cell to be filled.

Additional Instructions:

- Use digits 1-4.
- Output the <next_step> in the same format.
- Fill only one cell at each step.
- A partial but correct solution such as "3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1" is preferable to a full but incorrect one.


Please derive the first step, and return the step in the following JSON
format {"next_step": "<next_step>"}


> Finished chain.
3,1,4,2|1,4,3,2|2,1,4,3|4,3,2,1


> Entering new LLMChain chain...
Prompt after formatting:

For the given problem:


We have a 4x4 Sudoku puzzle "3,*,*,2|1,*,3,*|*,1,*,3|4,*,*,1" where * represents a cell to be filled.

Additional Instructions:

- Use digits 1-4.
- Output the <next_step> in the same format.
- Fill only one cell at each step.
- A partial but correct s

'3,4,1,2|1,2,3,4|2,1,4,3|4,3,2,1'